## Price Swing v1.0
## Test File - test code changes here first

This study is to analyze the price swings of the SP500 e-mini contract. Price swings move 
Final Project Proposal: “SP500 e-Mini Daytrading - Optimizing Pullback Entries” 

The SP500 e-Mini contract volatility has increased since Donald Trump became the U.S. President. The length of swings (measured in ticks) has increased and the number of swings per day has also increased. The question for the typical day trader is how/where do I enter a trade and what should my profit target be? A day trader needs some basic e-Mini swing descriptive statistics to plan and execute his day trading approach.

This proposed study will do 2 things:

1)	Develop descriptive statistics for SP500 e-mini 1 minute bar swings. (Note a swing high will be defined as high bar with 2 lower or equal bars before and after the swing high point and a similar approach for swing lows).  For example – how many ticks is a typical swing, how long does a swing go (in bars / time) before a pull back, does the swing have a breakout, if the swing has a breakout, how many bars does it go and how far past the previous swing high / low ? 

As shown in Figure 1 below, when the e-mini market seems to be moving in an “organized pattern, the market makes a series of higher highs and higher lows until the market reverses and makes a series of lower lows and lower highs. As a day trader, knowing that the down swings from 11:00 am to 12:00 pm EST typically average 20+ ticks in length down, 13 – 15 tick pull backs, and the breakouts below the previous swing lows only last for 1 – 2 bars is useful information. Figure 2 shows a similar structure.

Figure 3 is also a down trend however it’s more sideways trading. Seeing the swing lengths and number of bars in each swing (both with and against the trend) lets the trader develop an understanding of the current market environment and establish reasonable entry and stops.

2)	Optimize a pullback entry approach. Figures 1 – 3 all contain 3 exponential moving averages (ema). The blue line (21 ema) is a typical entry point and the red line (34 ema) and magenta (55 ema) show an approximate swing retracement area before the market continues in the larger trend direction.

My hypothesis is that a pull back entry at the 21 ema with a stop X ticks beyond the 55 ema can be profitable in the SP500 emini. I will need to leverage the existing backtesting code provided in our certificate program to develop pullback (i.e. limit entries) and then optimize these entries based upon the descriptive statistics analyses developed in step 1 above.

I will use the SP e-mini contract from Sept 2018 – Dec 2018 for my testing. 

Fig 1:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%201.png)

Fig 2:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%202.png)

Fig 3:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%203.png)

In [1]:
# Import Libraries
#
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly import __version__
print(__version__)

from datetime import datetime

# current date and time
now = datetime.now()
print(now)

3.7.0
2019-07-17 07:24:01.295344


In [2]:
# Reading ES 03 19 - Jan 1 - 18, 2019
# 
df = pd.read_csv(r"C:\Users\522147\Desktop\ES_03_19_2019_01_to_18_20secV2.csv", 
                    index_col=0, parse_dates=True, sep=",")
df.head(10)

,open,high,low,close
Date_Time,,,,
2019-01-01 23:00:00,2493.00,2493.00,2492.75,2493.00
2019-01-01 23:00:20,2493.25,2493.50,2493.25,2493.50
2019-01-01 23:00:40,2493.25,2493.25,2493.00,2493.25
2019-01-01 23:01:00,2493.50,2493.50,2493.25,2493.25
2019-01-01 23:01:20,2493.25,2493.50,2493.25,2493.25
2019-01-01 23:01:40,2493.25,2493.25,2492.75,2492.75
2019-01-01 23:02:00,2492.75,2492.75,2492.75,2492.75
2019-01-01 23:02:20,2492.50,2492.50,2492.00,2492.00
2019-01-01 23:02:40,2492.00,2492.00,2491.75,2491.75


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72361 entries, 2019-01-01 23:00:00 to 2019-01-18 17:00:00
Data columns (total 4 columns):
open     51953 non-null float64
high     51953 non-null float64
low      51953 non-null float64
close    51953 non-null float64
dtypes: float64(4)
memory usage: 2.8 MB


In [4]:
df.tail(10)

,open,high,low,close
Date_Time,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00


The 21 and 55 exponential moving average are key to this study. The 55 EMA will set the trend and pull backs to the 21 EMA will be used for entries

In [5]:
df['21ema'] = pd.Series.ewm(df['close'], span=21).mean()
df['55ema'] = pd.Series.ewm(df['close'], span=55).mean()

In [6]:
df.tail(10)

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259


## df_mini copy for testing sample

In [7]:
#
# Copying subset of data for testing
#
df_mini = df.iloc[-11000:].copy()    # increased from 3000 to 7000 to 11000

In [8]:
df_mini.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11000 entries, 2019-01-16 03:53:40 to 2019-01-18 17:00:00
Data columns (total 6 columns):
open     10369 non-null float64
high     10369 non-null float64
low      10369 non-null float64
close    10369 non-null float64
21ema    11000 non-null float64
55ema    11000 non-null float64
dtypes: float64(6)
memory usage: 601.6 KB


In [9]:
df_mini.tail(10)

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259


In [10]:
df_mini.head()

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-16 03:53:40,2614.50,2614.50,2614.25,2614.25,2614.344078,2614.128876
2019-01-16 03:54:00,2614.25,2614.50,2614.25,2614.25,2614.335525,2614.133202
2019-01-16 03:54:20,2614.25,2614.25,2614.00,2614.25,2614.327750,2614.137373
2019-01-16 03:54:40,2614.25,2614.50,2614.25,2614.50,2614.343409,2614.150324
2019-01-16 03:55:00,2614.50,2614.75,2614.25,2614.25,2614.334917,2614.153884


This study will set the trend based upon the current bar being "x points" above or below the 55 EMA. A trend is long when the high is the current
bar is "trend points (set at 1.5)" above the 55EMA. The trend is stays long until the low of a bar is "trend points" below the 55 EMA. The trend
then stays short until the long trend condition is set.

In [11]:
#  
# Creating trend change files - when price goes "trend_pts" above or below the 55EMA 
#
# DF [0]open, [1]high, [2]low, [3]close, [4]21ema, [5]55ema   
#
trend_change_up_plot = {}    # dict with the trend change up price to plot 
trend_change_dn_plot = {}    # dict with the trend change down price to plot
trend_change_flag = {}       # either 1 (long) or -1 (short)
prev_trend = 1
current_trend = 1
trend_pts = 1.5              # trend points determines the trend

for i, row in df_mini.iterrows():
    current_bar = i
    if df_mini.index.get_loc(i) == 0:          # initial run through - set entries for first bar
        trend_change_up_plot[i] = row[3]       # set initial dict to have close value
        trend_change_flag[i] = 1     

##--------- Down Trend Change --------------------------------------------------------------------------------#    
    if current_trend == 1:                     # assume initial upswing - check 
        if row[2] <= row[5] - trend_pts:       # if low < 55EMA - trend points
            prev_trend = current_trend
            trend_change_dn_plot[i] = row[2]   # set initial dict to have low
            trend_change_flag[i] =  -1   
            current_trend = -1
    
##--------- Up Trend Change --------------------------------------------------------------------------------#    
    if current_trend == -1:                    # if current trend is down 
        if row[1] >= row[5] + trend_pts:       # if high > 55EMA + trend points
            prev_trend = current_trend
            trend_change_up_plot[i] = row[1]
            trend_change_flag[i] =  1  
            current_trend = 1

In [12]:
#
# Convert Trend Change up/down and trend change flags to dataframes. These data frames will be used to plot the trend changes on the price chart
#
df_trend_change_up_plot = pd.DataFrame.from_dict(trend_change_up_plot,orient='index',columns=['Price'])
df_trend_change_up_plot.index.names= ['Date_Time']  

df_trend_change_dn_plot = pd.DataFrame.from_dict(trend_change_dn_plot,orient='index',columns=['Price'])
df_trend_change_dn_plot.index.names= ['Date_Time']  

df_trend_change_flag = pd.DataFrame.from_dict(trend_change_flag,orient='index',columns=['trend'])
df_trend_change_flag.index.names= ['Date_Time']  

The trend flag dataframe is merged with the price chart data to establish price swings occuring with and against the trend

In [13]:
# 
# Merge trend change dataframe with the price data and reset the index
#
df_mini = pd.merge_ordered(df_mini, df_trend_change_flag, on='Date_Time',how='left',fill_method='ffill')
df_mini.set_index('Date_Time',inplace=True)

A new "up" price swing occurs when the current bar high exceeded the highest hi of the past 5 previous bars. A new "down" swing occurs
when the low of the current bar exceeds the low of the past 5 lowest bars.

The variables below hold the rolling 5 bar max highs and 5 bar min lows and the current bar crosses these values 

In [14]:
#
# set 5 previous bars max hi and min low and when the current bar crosses them
#

df_mini['trail_hi'] = df_mini['high'].rolling(5,min_periods=1).max()
df_mini['trail_hix'] = df_mini['high'] - df_mini['trail_hi'].shift(1)
df_mini['trail_lo'] = df_mini['low'].rolling(5,min_periods=1).min()
df_mini['trail_lox'] = df_mini['trail_lo'].shift(1) - df_mini['low']    
df_mini['hour'] = df_mini.index.hour
df_mini.tail(10)

,open,high,low,close,21ema,55ema,trend,trail_hi,trail_hix,trail_lo,trail_lox,hour
Date_Time,,,,,,,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863,1,2677.00,-0.50,2675.50,-1.25,16
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017,1,2677.00,-0.50,2675.50,-0.50,16
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751,1,2676.75,-0.25,2676.00,-0.50,16
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805,1,2676.75,0.00,2676.00,-0.50,16
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664,1,2677.25,0.50,2676.00,-0.50,16
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457,1,2677.25,0.00,2676.00,-0.25,16
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798,1,2677.50,0.25,2676.00,-0.75,16
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686,1,2677.50,-0.50,2676.25,-0.25,16
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259,1,2677.50,-0.25,2676.25,0.00,16


The code below is the main logic for calculating swings and their length in price bars and points

The logic creates a separate file for up swings and down swings to be plotted on the chart separately

In [15]:
# Using iterrows to calc swings  -   latest version 2019-07-16
# 
# data file:
# Index - Date/Time  [0]open, [1]high, [2]low, [3]close, [4]21ema, [5]55ema, [6]trend, [7]trail_hi, [8]trail_hix, [9]trail_lo, [10]trail_lox 
#
# initial var setup
up_down = 0
swing = 0
diff = 2.0
upswing_list = []
downswing_list = []
all_swing_list = []

debug_print = 1

print(datetime.now())
for i, row in df_mini.iterrows():
    if df_mini.index.get_loc(i) == 0:          # initial run through - set entries for first bar
        prev_swing_h = row[1]
        prev_swing_h_loc = i
        prev_swing_h_barnum = 0
        swing_h = row[1]
        swing_hl = row[2]
        swing_h_loc = i
        swing_h_barnum = 0
        swing_h_trend = 1
        prev_swing_l = row[2]
        prev_swing_l_loc = i
        prev_swing_l_barnum = 0 
        swing_l = row[2]
        swing_lh = row[1]
        swing_l_loc = i
        swing_l_barnum = 0
        swing_l_trend = 1
        up_pts = 0
        prev_up_pts = 0 
        dn_pts = 0
        prev_dn_pts = 0
        swing_h_ema21=0
        swing_l_ema21=0
        swing_h_ema55=0
        swing_l_ema55=0
        
    ##--------- UP SWING --------------------------------------------------------------------------------#    
    if swing == 1:                 # assume initial upswing - check 
        if row[1] > swing_h:       # if high > crrent hi, calc upswing stats,  make it >, not >= so the first peak is swing hi
            swing_h = row[1]
            swing_hl = row[2]
            swing_h_loc = i
            swing_h_barnum = df_mini.index.get_loc(i)
            up_pts = swing_h - prev_swing_l
            ema21 = row[4]
            ema55 = row[5]
                                        
        if row[10] > 0:   # In Up swing and Down swing hit (lowest x lows crossed) 
            up_pts = swing_h - prev_swing_l
            up_swing_len = swing_h_barnum - prev_swing_l_barnum
            if round(dn_pts,2) > 0.00 and round(prev_up_pts,2) > 0.00:
                swing_retrace_perc = dn_pts / prev_up_pts
            else:
                swing_retrace_perc = 1.0
            swing_breakout_pts = round(swing_h - prev_swing_h,2)
          
            if round(swing_breakout_pts,2) == 0.00:
                swing_breakout_type = 'EH'
                swing_breakout = 1.0
             #   print(f'$$$$ Up Swing EL rounded swing breakout pts {round(swing_breakout_pts,2)} EL Breakout type {swing_breakout_type}')
            elif round(swing_breakout_pts,2) > 0.00:
                swing_breakout_type = 'HH'
                swing_breakout = 1.0
             #   print(f'$$$$ Up SwingLL rounded swing breakout pts {round(swing_breakout_pts,2)} LL Breakout type {swing_breakout_type}')
            else:
                swing_breakout_type = 'LH'
                swing_breakout = 0.0
                  
            prev_up_pts = up_pts
            prev_swing_h = swing_h               # hold prev swing high after new formed down swing
            prev_swing_hl = swing_hl
            prev_swing_h_loc = swing_h_loc       #
            prev_swing_h_barnum = swing_h_barnum #  
            #up_swing_len = 99
            swing = -1                           # now down swing
            swing_l = row[2]                     # reset low to current low
            swing_l_loc = i
            swing_l_barnum = df_mini.index.get_loc(i)
            swing_l_trend = row[6]
            swing_l_ema21 = row[4]
            swing_l_ema55 = row[5]
            
            upswing_list.append({'x0': str(swing_h_loc), 'high':prev_swing_h, 'low':prev_swing_hl})
            if swing_h_trend == 1:
                swing_type = 'LWTr' 
            else:
                swing_type = 'LATr'
            all_swing_list.append({'Date_Time':prev_swing_l_loc, 'trend':swing_h_trend, 'swing':1, 'points':up_pts,'swing_len':up_swing_len,'swing_type':swing_type,'retrace':round(swing_retrace_perc,2), \
                                   'Brkout':round(swing_breakout,1),'BrkType': swing_breakout_type, 'Brkpts':round(swing_breakout_pts,2),'21EMA': round(swing_h_ema21,2),'55E':round(swing_h_ema55,2)})
    ##--------- DOWN SWING ------------------------------------------------------------------------------#    
    if swing == -1:                 # if current down swing 
        if row[2] < swing_l:        # new swing low - update current swing
            swing_l = row[2]
            swing_lh = row[1]
            swing_l_loc = i
            swing_l_barnum = df_mini.index.get_loc(i)
            dn_pts = prev_swing_h - swing_l
            ema21 = row[4]
            ema55 = row[5]
                        
        if row[8] > 0:   # In Down swing and Up swing hit (highest x highs crossed)
            dn_pts = prev_swing_h - swing_l
            dn_swing_len = swing_l_barnum - prev_swing_h_barnum
            if round(up_pts,2) > 0.00 and round(prev_dn_pts,2) > 0.00:
                swing_retrace_perc = up_pts / prev_dn_pts
            else:
                swing_retrace_perc = 1.0
            
            swing_breakout_pts = round(prev_swing_l - swing_l,2)
                        
            if round(swing_breakout_pts,2) == 0.00:
                swing_breakout_type = 'EL'
                swing_breakout = 1.0
            elif round(swing_breakout_pts,2) > 0.00:
                swing_breakout_type = 'LL'
                swing_breakout = 1.0
            else:
                swing_breakout_type = 'HL'
                swing_breakout = 0.0
  
            prev_dn_pts = dn_pts
            prev_swing_l = swing_l               # hold prev swing low after new formed up swing
            prev_swing_lh = swing_lh
            prev_swing_l_loc = swing_l_loc       #
            prev_swing_l_barnum = swing_l_barnum
            swing = 1                            # now down swing
            swing_h = row[1]                     # reset high to current high
            swing_h_loc = i
            swing_h_barnum = df_mini.index.get_loc(i)
            swing_h_trend = row[6]
            swing_h_ema21 = row[4]
            swing_h_ema55 = row[5]
            
            downswing_list.append({'x0': str(swing_l_loc), 'high':prev_swing_lh, 'low':prev_swing_l})
            if swing_l_trend == -1:
                swing_type = 'SWTr' 
            else:
                swing_type = 'SATr'
            
            all_swing_list.append({'Date_Time':prev_swing_h_loc, 'trend':swing_l_trend, 'swing':-1, 'points':dn_pts,'swing_len':dn_swing_len,'swing_type':swing_type,'retrace':round(swing_retrace_perc,2), \
                                   'Brkout':round(swing_breakout,1),'BrkType': swing_breakout_type, 'Brkpts':round(swing_breakout_pts,2),'21EMA': round(swing_l_ema21,2),'55E':round(swing_l_ema55,2)})
    ##------

    if df_mini.index.get_loc(i) == 0:     # set up after skipping over initial swing logic
        swing =1
        print(f'Set Swing: {swing}')

2019-07-17 07:26:29.386494
Set Swing: 1


In [16]:
#
# print df to text file for testing
#
print(type(all_swing_list))
print(len(all_swing_list))

with open("all_swing_list.txt", "w") as output:
    output.write(str(datetime.now())+"\n")
    for i in range(0,len(all_swing_list)):
        #print('in print loop')
        output.write(str(all_swing_list[i])+"\n")

df_mini.to_excel("all_bars_list.xlsx")  

<class 'list'>
956


In [17]:
print(len(upswing_list))
#print(upswing_list)

478


In [18]:
print(len(downswing_list))
#print(downswing_list)

478


In [19]:
# Converting DF index to string for plotly

df_mini_str = []
for i, row in df_mini.iterrows():
    df_mini_str.append(str(i))

In [20]:
# 
# Convert index to string for plotly. PLotly does not plot the time of a bar correctly using the date time stamp index but it does 
# plot correctly with the data time stamp converted to a string
#
df_trend_change_up_plot_str = []
df_trend_change_dn_plot_str = []
 
for i, row in df_trend_change_up_plot.iterrows():
    df_trend_change_up_plot_str.append(str(i))

for i, row in df_trend_change_dn_plot.iterrows():
    df_trend_change_dn_plot_str.append(str(i))

In [21]:
#
# creates line plot list for swing start - plotly needs a dictionary built for each plot
#
blue_dict = {'line':{'color': 'blue', 'width': 1}}
red_dict = {'line':{'color': 'red', 'width': 1}}

line_plot_list = []
zlist = []

def list_line_plot(list_name,color_plot):

    ctr = 0
    z1 = []
    z2 = []
    z3 = []
    z4 = []
    z5 = []
    z6 = []
    z7 = []
    z8 = []
    z9 = []
    z10 = []
    z11 = []

    for item in list_name:
        z1.append('type')
        z2.append('line')
        z3.append('x0')
        z4.append(item["x0"])      # bar date
        z5.append('y0')
        if item["low"] > 0:
            z6.append(item["low"] - 8)
            z10.append(item["low"] - 1)
        else:
            z6.append(2465.00)  
            z10.append(2470.00)  
        z7.append('x1')
        z8.append(item["x0"])      # bar date
        z9.append('y1')
        z11.append('line')

    for i1, i2, i3, i4, i5, i6, i7, i8, i9, i10 in zip(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10):
        zlist.append({i1: i2, i3:i4, i5: i6, i7:i8, i9: i10})

    for item in zlist:
        item.update(color_plot)
    
    return zlist   # zlist is copied to line_list_plot
#---------------- End of List_Line_PLot function    

# creete downswing plots
list_line_plot(downswing_list,blue_dict)

# append down swing plots to the line_plot_list
line_plot_list = line_plot_list + zlist

# append upswing swing plots to the line_plot_list
zlist = []
list_line_plot(upswing_list,red_dict)
line_plot_list = line_plot_list + zlist

print_list = 0
if print_list == 1:
    with open("line_plot_list.txt", "w") as output:
        output.write(str(datetime.now())+"\n")
        for i in range(0,len(line_plot_list)):
            #print('in print loop')
            output.write(str(line_plot_list[i])+"\n")

This plotly charts the price data, the trend changes (green dot is a new up trend and a red dot is a new down trend), and the price swings.

The start of an up price swing is shown by a blue line and the start of a down swing is shown by a red line

The chart plots to html as this data is easier to view on a larger plot

In [22]:
# 
# PLotly with multiple plots 
#

trace1 = go.Ohlc(x=df_mini_str, 
                open=df_mini['open'],
                high=df_mini['high'],
                low=df_mini['low'],
                close=df_mini['close'],
                increasing=dict(line=dict(color= '#33CC33')),
                decreasing=dict(line=dict(color= '#CC3300')))

trace2 = go.Scatter(x=df_mini_str,
                            y=df_mini['21ema'],
                            mode='lines',name='21ema')
trace3 = go.Scatter(x=df_mini_str,
                            y=df_mini['55ema'],
                            mode='lines',name='55ema')

trace4 = go.Scatter(x=df_trend_change_up_plot_str,
                            y=df_trend_change_up_plot['Price'],
                            mode='markers',
                            marker = dict(
                            symbol = 'diamond',
                            #    symbol = 'triangle-up',
                            color = '#00ff00',
                            size = 10,
                            line = dict(width = 1)),
                            name='Up Trend')

trace5 = go.Scatter(x=df_trend_change_dn_plot_str,
                            y=df_trend_change_dn_plot['Price'],
                            mode='markers',
                            marker = dict(
                            symbol = 'diamond',
                            #    symbol = 'triangle-up',
                            color = '#ff0000',
                            size = 10,
                            line = dict(width = 1)),
                            name='Dn Trend')

layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True),
                shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
data = [trace1,trace2,trace3,trace4,trace5]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='styled_ohlc.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\styled_ohlc.html'

# Separate all swing files into separate swing types (up and down)

In [23]:
# pring off swings

for i in range(0,5):
    print(all_swing_list[i])

{'Date_Time': Timestamp('2019-01-16 03:53:40'), 'trend': 1, 'swing': 1, 'points': 0.25, 'swing_len': 0, 'swing_type': 'LWTr', 'retrace': 1.0, 'Brkout': 1.0, 'BrkType': 'EH', 'Brkpts': 0.0, '21EMA': 0, '55E': 0}
{'Date_Time': Timestamp('2019-01-16 03:53:40'), 'trend': 1.0, 'swing': -1, 'points': 0.5, 'swing_len': 2, 'swing_type': 'SATr', 'retrace': 1.0, 'Brkout': 1.0, 'BrkType': 'LL', 'Brkpts': 0.25, '21EMA': 2614.33, '55E': 2614.14}
{'Date_Time': Timestamp('2019-01-16 03:54:20'), 'trend': 1.0, 'swing': 1, 'points': 0.75, 'swing_len': 2, 'swing_type': 'LWTr', 'retrace': 2.0, 'Brkout': 1.0, 'BrkType': 'HH', 'Brkpts': 0.25, '21EMA': 2614.33, '55E': 2614.15}
{'Date_Time': Timestamp('2019-01-16 03:55:00'), 'trend': 1.0, 'swing': -1, 'points': 1.75, 'swing_len': 4, 'swing_type': 'SATr', 'retrace': 1.5, 'Brkout': 1.0, 'BrkType': 'LL', 'Brkpts': 1.0, '21EMA': 2614.28, '55E': 2614.14}
{'Date_Time': Timestamp('2019-01-16 03:56:20'), 'trend': 1.0, 'swing': 1, 'points': 1.5, 'swing_len': 7, 'swing

In [24]:
# Transform output finle into dataframe
test_df = pd.DataFrame(all_swing_list)
test_df.set_index('Date_Time',inplace=True)
test_df.head()

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend
Date_Time,,,,,,,,,,,
2019-01-16 03:53:40,0.00,0.00,EH,1.0,0.00,0.25,1.00,1,0,LWTr,1.0
2019-01-16 03:53:40,2614.33,2614.14,LL,1.0,0.25,0.50,1.00,-1,2,SATr,1.0
2019-01-16 03:54:20,2614.33,2614.15,HH,1.0,0.25,0.75,2.00,1,2,LWTr,1.0
2019-01-16 03:55:00,2614.28,2614.14,LL,1.0,1.00,1.75,1.50,-1,4,SATr,1.0
2019-01-16 03:56:20,2613.97,2614.03,LH,0.0,-0.25,1.50,2.33,1,7,LWTr,1.0


In [25]:
test_df['BrkType'].value_counts()

HH    223
HL    220
LL    196
LH    193
EL     62
EH     62
Name: BrkType, dtype: int64

In [26]:
test_df['hour'] = test_df.index.hour

In [27]:
print(test_df.head(10))

                       21EMA      55E BrkType  Brkout  Brkpts  points  \
Date_Time                                                               
2019-01-16 03:53:40     0.00     0.00      EH     1.0    0.00    0.25   
2019-01-16 03:53:40  2614.33  2614.14      LL     1.0    0.25    0.50   
2019-01-16 03:54:20  2614.33  2614.15      HH     1.0    0.25    0.75   
2019-01-16 03:55:00  2614.28  2614.14      LL     1.0    1.00    1.75   
2019-01-16 03:56:20  2613.97  2614.03      LH     0.0   -0.25    1.50   
2019-01-16 03:58:40  2614.03  2614.04      HL     0.0   -0.50    1.00   
2019-01-16 04:00:00  2614.12  2614.08      HH     1.0    0.50    1.50   
2019-01-16 04:02:00  2614.04  2614.06      EL     1.0    0.00    1.50   
2019-01-16 04:04:40  2614.11  2614.08      HH     1.0    0.25    1.75   
2019-01-16 04:05:00  2614.27  2614.17      LL     1.0    1.25    3.00   

                     retrace  swing  swing_len swing_type  trend  hour  
Date_Time                                         

In [28]:
test_df.to_excel("all_swings_breakouts_list.xlsx")  # export to excel

In [29]:
#
# Categorize all trading hours as "Core" (9 am EST to 3 pm EST) and all other times as "Noncore"
#
lst = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
lst2 = ['noncore','noncore', 'noncore', 'noncore', 'noncore', 'noncore', 'noncore', 'noncore', 'noncore', 'core', 'core', 'core', 'core', 'core', 'core', 'core', 'noncore', \
        'noncore', 'noncore', 'noncore', 'noncore', 'noncore', 'noncore', 'noncore', 'noncore']
test_core = pd.DataFrame(list(zip(lst, lst2)), columns = ['hour','hour_type'])

In [30]:
# Merge Core / Noncore dataframe with swing data
test_df = pd.merge(test_df,test_core,on='hour',how='left')
test_df.head()

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
0,0.00,0.00,EH,1.0,0.00,0.25,1.00,1,0,LWTr,1.0,3,noncore
1,2614.33,2614.14,LL,1.0,0.25,0.50,1.00,-1,2,SATr,1.0,3,noncore
2,2614.33,2614.15,HH,1.0,0.25,0.75,2.00,1,2,LWTr,1.0,3,noncore
3,2614.28,2614.14,LL,1.0,1.00,1.75,1.50,-1,4,SATr,1.0,3,noncore
4,2613.97,2614.03,LH,0.0,-0.25,1.50,2.33,1,7,LWTr,1.0,3,noncore


In [31]:
test_df[['points','swing_len']].describe(percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

,points,swing_len
count,956.000000,956.000000
mean,2.155073,11.488494
std,1.603521,12.188522
min,0.250000,0.000000
10%,0.750000,3.000000
20%,1.000000,4.000000
30%,1.250000,6.000000
40%,1.500000,7.000000
50%,1.750000,9.000000
75%,2.750000,15.000000


In [32]:
test_df.head()

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
0,0.00,0.00,EH,1.0,0.00,0.25,1.00,1,0,LWTr,1.0,3,noncore
1,2614.33,2614.14,LL,1.0,0.25,0.50,1.00,-1,2,SATr,1.0,3,noncore
2,2614.33,2614.15,HH,1.0,0.25,0.75,2.00,1,2,LWTr,1.0,3,noncore
3,2614.28,2614.14,LL,1.0,1.00,1.75,1.50,-1,4,SATr,1.0,3,noncore
4,2613.97,2614.03,LH,0.0,-0.25,1.50,2.33,1,7,LWTr,1.0,3,noncore


In [33]:
df_LWTr = test_df[test_df['swing_type'] == 'LWTr']             # LWTr - Long (Up) Swing with Trend
df_LWTr.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
0,0.00,0.00,EH,1.0,0.00,0.25,1.00,1,0,LWTr,1.0,3,noncore
2,2614.33,2614.15,HH,1.0,0.25,0.75,2.00,1,2,LWTr,1.0,3,noncore
4,2613.97,2614.03,LH,0.0,-0.25,1.50,2.33,1,7,LWTr,1.0,3,noncore
6,2614.12,2614.08,HH,1.0,0.50,1.50,0.67,1,6,LWTr,1.0,4,noncore
8,2614.11,2614.08,HH,1.0,0.25,1.75,1.00,1,1,LWTr,1.0,4,noncore
10,2612.91,2613.42,LH,0.0,-2.00,1.00,1.71,1,8,LWTr,1.0,4,noncore
28,2612.53,2611.88,HH,1.0,0.75,1.50,0.33,1,6,LWTr,1.0,5,noncore
30,2613.09,2612.53,EH,1.0,0.00,1.25,0.83,1,11,LWTr,1.0,5,noncore
32,2612.72,2612.67,LH,0.0,-1.25,0.50,1.40,1,10,LWTr,1.0,5,noncore
66,2610.74,2609.92,HH,1.0,1.00,3.00,0.57,1,27,LWTr,1.0,7,noncore


In [34]:
df_LWTr_core = df_LWTr[df_LWTr['hour_type'] == 'core']         # LWTr - Long (Up) Swing with Trend for Core Hours
df_LWTr_core.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
86,2614.54,2613.89,HH,1.0,1.25,2.00,0.27,1,9,LWTr,1.0,9,core
88,2614.68,2614.76,LH,0.0,-0.25,2.25,1.25,1,15,LWTr,1.0,9,core
90,2614.38,2614.58,EH,1.0,0.00,2.50,1.11,1,0,LWTr,1.0,9,core
92,2614.79,2614.71,HH,1.0,3.00,6.25,1.30,1,25,LWTr,1.0,9,core
94,2617.81,2616.64,HH,1.0,5.00,6.50,0.24,1,21,LWTr,1.0,9,core
96,2620.79,2619.75,LH,0.0,-2.00,2.25,0.65,1,11,LWTr,1.0,9,core
98,2621.19,2620.53,HH,1.0,2.00,3.75,0.78,1,21,LWTr,1.0,10,core
100,2622.96,2622.40,LH,0.0,-0.50,1.25,0.47,1,3,LWTr,1.0,10,core
102,2622.98,2622.56,HH,1.0,0.25,1.50,1.00,1,5,LWTr,1.0,10,core
110,2618.09,2617.61,HH,1.0,0.25,1.50,0.19,1,4,LWTr,1.0,10,core


In [35]:
df_LWTr_noncore = df_LWTr[df_LWTr['hour_type'] == 'noncore']   # LWTr - Long (Up) Swing with Trend for Noncore Hours
df_LWTr_noncore.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
0,0.00,0.00,EH,1.0,0.00,0.25,1.00,1,0,LWTr,1.0,3,noncore
2,2614.33,2614.15,HH,1.0,0.25,0.75,2.00,1,2,LWTr,1.0,3,noncore
4,2613.97,2614.03,LH,0.0,-0.25,1.50,2.33,1,7,LWTr,1.0,3,noncore
6,2614.12,2614.08,HH,1.0,0.50,1.50,0.67,1,6,LWTr,1.0,4,noncore
8,2614.11,2614.08,HH,1.0,0.25,1.75,1.00,1,1,LWTr,1.0,4,noncore
10,2612.91,2613.42,LH,0.0,-2.00,1.00,1.71,1,8,LWTr,1.0,4,noncore
28,2612.53,2611.88,HH,1.0,0.75,1.50,0.33,1,6,LWTr,1.0,5,noncore
30,2613.09,2612.53,EH,1.0,0.00,1.25,0.83,1,11,LWTr,1.0,5,noncore
32,2612.72,2612.67,LH,0.0,-1.25,0.50,1.40,1,10,LWTr,1.0,5,noncore
66,2610.74,2609.92,HH,1.0,1.00,3.00,0.57,1,27,LWTr,1.0,7,noncore


In [36]:
df_LATr = test_df[test_df['swing_type'] == 'LATr']             # LATr - Long (Up) Swing against Trend
df_LATr.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
12,2611.04,2612.01,LH,0.0,-1.75,1.50,3.25,1,5,LATr,-1.0,4,noncore
14,2610.62,2611.34,LH,0.0,-0.75,0.75,1.00,1,6,LATr,-1.0,4,noncore
16,2610.37,2611.03,HH,1.0,1.00,2.25,1.67,1,15,LATr,-1.0,4,noncore
18,2610.64,2610.86,LH,0.0,-0.50,1.25,0.78,1,4,LATr,-1.0,4,noncore
20,2610.48,2610.71,HH,1.0,1.00,2.50,1.20,1,9,LATr,-1.0,4,noncore
22,2610.59,2610.75,LH,0.0,-0.75,1.75,1.00,1,12,LATr,-1.0,4,noncore
24,2610.90,2610.86,HH,1.0,0.50,1.75,0.71,1,9,LATr,-1.0,5,noncore
26,2611.24,2611.07,HH,1.0,1.25,2.25,0.57,1,22,LATr,-1.0,5,noncore
34,2611.24,2611.67,LH,0.0,-1.25,1.00,4.50,1,12,LATr,-1.0,5,noncore
36,2610.96,2611.37,LH,0.0,-0.25,0.75,1.00,1,7,LATr,-1.0,5,noncore


In [37]:
df_LATr_core = df_LATr[df_LATr['hour_type'] == 'core']         # LWAr - Long (Up) Swing against Trend for Core Hours
df_LATr_core.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
104,2621.54,2621.96,LH,0.0,-1.75,1.75,2.33,1,13,LATr,-1.0,10,core
106,2615.81,2617.77,LH,0.0,-6.25,2.50,5.00,1,7,LATr,-1.0,10,core
108,2615.14,2616.90,HH,1.0,3.50,6.75,1.30,1,23,LATr,-1.0,10,core
114,2615.72,2616.73,LH,0.0,-3.00,3.75,3.00,1,12,LATr,-1.0,11,core
116,2616.18,2616.50,HH,1.0,4.75,6.25,0.40,1,24,LATr,-1.0,11,core
126,2616.59,2618.01,LH,0.0,-3.75,3.75,5.00,1,10,LATr,-1.0,11,core
128,2617.07,2617.59,EH,1.0,0.00,2.75,0.73,1,9,LATr,-1.0,12,core
130,2617.56,2617.58,HH,1.0,1.25,3.50,0.82,1,12,LATr,-1.0,12,core
132,2616.92,2617.36,LH,0.0,-0.25,5.50,1.64,1,8,LATr,-1.0,12,core
140,2617.37,2617.77,HH,1.0,1.00,4.00,1.50,1,17,LATr,-1.0,12,core


In [41]:
df_LATr_noncore = df_LATr[df_LATr['hour_type'] == 'noncore']  # LATr - Long (Up) Swing against Trend for Noncore Hours
df_LATr_noncore.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
12,2611.04,2612.01,LH,0.0,-1.75,1.50,3.25,1,5,LATr,-1.0,4,noncore
14,2610.62,2611.34,LH,0.0,-0.75,0.75,1.00,1,6,LATr,-1.0,4,noncore
16,2610.37,2611.03,HH,1.0,1.00,2.25,1.67,1,15,LATr,-1.0,4,noncore
18,2610.64,2610.86,LH,0.0,-0.50,1.25,0.78,1,4,LATr,-1.0,4,noncore
20,2610.48,2610.71,HH,1.0,1.00,2.50,1.20,1,9,LATr,-1.0,4,noncore
22,2610.59,2610.75,LH,0.0,-0.75,1.75,1.00,1,12,LATr,-1.0,4,noncore
24,2610.90,2610.86,HH,1.0,0.50,1.75,0.71,1,9,LATr,-1.0,5,noncore
26,2611.24,2611.07,HH,1.0,1.25,2.25,0.57,1,22,LATr,-1.0,5,noncore
34,2611.24,2611.67,LH,0.0,-1.25,1.00,4.50,1,12,LATr,-1.0,5,noncore
36,2610.96,2611.37,LH,0.0,-0.25,0.75,1.00,1,7,LATr,-1.0,5,noncore


In [38]:
df_SWTr = test_df[test_df['swing_type'] == 'SWTr']            # SWTr - Short (Down) Swing with Trend 
df_SWTr.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
13,2611.15,2611.85,EL,1.0,0.00,1.50,0.46,-1,14,SWTr,-1.0,4,noncore
15,2610.48,2611.21,LL,1.0,0.50,1.25,0.50,-1,3,SWTr,-1.0,4,noncore
17,2610.91,2611.03,HL,0.0,-0.50,1.75,1.80,-1,13,SWTr,-1.0,4,noncore
19,2610.50,2610.76,LL,1.0,0.25,1.50,0.71,-1,5,SWTr,-1.0,4,noncore
21,2610.99,2610.90,EL,1.0,0.00,2.50,1.67,-1,13,SWTr,-1.0,4,noncore
23,2610.94,2610.87,HL,0.0,-0.50,1.25,0.70,-1,8,SWTr,-1.0,4,noncore
25,2611.17,2611.02,HL,0.0,-0.75,1.00,1.40,-1,5,SWTr,-1.0,5,noncore
35,2611.18,2611.58,EL,1.0,0.00,1.00,0.44,-1,6,SWTr,-1.0,5,noncore
37,2610.92,2611.29,LL,1.0,1.00,1.75,0.75,-1,13,SWTr,-1.0,5,noncore
39,2610.04,2610.52,LL,1.0,2.00,3.00,0.57,-1,42,SWTr,-1.0,5,noncore


In [39]:
df_SWTr_core = df_SWTr[df_SWTr['hour_type'] == 'core']        # SWTr - Short (Down) Swing with Trend for Core Hours 
df_SWTr_core.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
91,2614.35,2614.56,LL,1.0,0.75,3.25,1.00,-1,1,SWTr,-1.0,9,core
105,2621.42,2621.88,LL,1.0,7.00,8.75,0.50,-1,46,SWTr,-1.0,10,core
107,2615.53,2617.38,LL,1.0,0.75,3.25,0.29,-1,3,SWTr,-1.0,10,core
115,2615.98,2616.47,HL,0.0,-2.25,1.50,0.56,-1,3,SWTr,-1.0,11,core
127,2617.29,2617.84,HL,0.0,-1.00,2.75,0.50,-1,10,SWTr,-1.0,12,core
129,2617.35,2617.60,HL,0.0,-0.50,2.25,1.00,-1,7,SWTr,-1.0,12,core
149,2619.31,2619.41,HL,0.0,-0.25,2.25,0.67,-1,9,SWTr,-1.0,13,core
151,2619.08,2619.28,LL,1.0,0.50,2.00,0.67,-1,2,SWTr,-1.0,13,core
153,2619.65,2619.46,HL,0.0,-1.75,1.50,1.62,-1,2,SWTr,-1.0,13,core
155,2619.99,2619.69,HL,0.0,-0.25,1.25,1.00,-1,2,SWTr,-1.0,13,core


In [40]:
df_SWTr_noncore = df_SWTr[df_SWTr['hour_type'] == 'noncore']  # SWTr - Short (Down) Swing with Trend for Noncore Hours 
df_SWTr_noncore.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
13,2611.15,2611.85,EL,1.0,0.00,1.50,0.46,-1,14,SWTr,-1.0,4,noncore
15,2610.48,2611.21,LL,1.0,0.50,1.25,0.50,-1,3,SWTr,-1.0,4,noncore
17,2610.91,2611.03,HL,0.0,-0.50,1.75,1.80,-1,13,SWTr,-1.0,4,noncore
19,2610.50,2610.76,LL,1.0,0.25,1.50,0.71,-1,5,SWTr,-1.0,4,noncore
21,2610.99,2610.90,EL,1.0,0.00,2.50,1.67,-1,13,SWTr,-1.0,4,noncore
23,2610.94,2610.87,HL,0.0,-0.50,1.25,0.70,-1,8,SWTr,-1.0,4,noncore
25,2611.17,2611.02,HL,0.0,-0.75,1.00,1.40,-1,5,SWTr,-1.0,5,noncore
35,2611.18,2611.58,EL,1.0,0.00,1.00,0.44,-1,6,SWTr,-1.0,5,noncore
37,2610.92,2611.29,LL,1.0,1.00,1.75,0.75,-1,13,SWTr,-1.0,5,noncore
39,2610.04,2610.52,LL,1.0,2.00,3.00,0.57,-1,42,SWTr,-1.0,5,noncore


In [41]:
df_SATr = test_df[test_df['swing_type'] == 'SATr']            # SATr - Short (Down) Swing against Trend
df_SATr.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
1,2614.33,2614.14,LL,1.0,0.25,0.50,1.00,-1,2,SATr,1.0,3,noncore
3,2614.28,2614.14,LL,1.0,1.00,1.75,1.50,-1,4,SATr,1.0,3,noncore
5,2614.03,2614.04,HL,0.0,-0.50,1.00,0.86,-1,4,SATr,1.0,3,noncore
7,2614.04,2614.06,EL,1.0,0.00,1.50,1.50,-1,8,SATr,1.0,4,noncore
9,2614.27,2614.17,LL,1.0,1.25,3.00,1.17,-1,37,SATr,1.0,4,noncore
11,2612.82,2613.28,LL,1.0,2.25,3.25,0.33,-1,20,SATr,1.0,4,noncore
27,2612.43,2611.82,HL,0.0,-1.50,0.75,2.25,-1,2,SATr,1.0,5,noncore
29,2612.97,2612.23,HL,0.0,-0.25,1.25,2.00,-1,5,SATr,1.0,5,noncore
31,2613.21,2612.75,LL,1.0,0.50,1.75,1.00,-1,9,SATr,1.0,5,noncore
33,2612.67,2612.65,LL,1.0,1.75,2.25,0.29,-1,35,SATr,1.0,5,noncore


In [42]:
df_SATr_core = df_SATr[df_SATr['hour_type'] == 'core']        # SATr - Short (Down) Swing against Trend for Core Hours
df_SATr_core.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
85,2614.31,2613.58,HL,0.0,-2.00,0.75,1.83,-1,13,SATr,1.0,9,core
87,2615.33,2614.69,LL,1.0,0.50,2.50,2.67,-1,43,SATr,1.0,9,core
89,2614.38,2614.58,LL,1.0,0.25,2.50,0.90,-1,0,SATr,1.0,9,core
93,2617.63,2616.44,HL,0.0,-4.75,1.50,1.92,-1,5,SATr,1.0,9,core
95,2621.00,2619.17,HL,0.0,-2.25,4.25,4.33,-1,17,SATr,1.0,9,core
97,2621.08,2620.47,HL,0.0,-0.50,1.75,0.53,-1,11,SATr,1.0,9,core
99,2622.93,2622.01,HL,0.0,-2.00,1.75,2.14,-1,17,SATr,1.0,10,core
101,2622.90,2622.45,EL,1.0,0.00,1.25,0.71,-1,4,SATr,1.0,10,core
103,2622.77,2622.54,LL,1.0,2.00,3.50,1.20,-1,15,SATr,1.0,10,core
109,2617.86,2617.47,HL,0.0,-5.50,1.25,2.08,-1,4,SATr,1.0,10,core


In [43]:
df_SATr_noncore = df_SATr[df_SATr['hour_type'] == 'noncore']  # SATr - Short (Down) Swing against Trend for Noncore Hours
df_SATr_noncore.head(10)

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
1,2614.33,2614.14,LL,1.0,0.25,0.50,1.00,-1,2,SATr,1.0,3,noncore
3,2614.28,2614.14,LL,1.0,1.00,1.75,1.50,-1,4,SATr,1.0,3,noncore
5,2614.03,2614.04,HL,0.0,-0.50,1.00,0.86,-1,4,SATr,1.0,3,noncore
7,2614.04,2614.06,EL,1.0,0.00,1.50,1.50,-1,8,SATr,1.0,4,noncore
9,2614.27,2614.17,LL,1.0,1.25,3.00,1.17,-1,37,SATr,1.0,4,noncore
11,2612.82,2613.28,LL,1.0,2.25,3.25,0.33,-1,20,SATr,1.0,4,noncore
27,2612.43,2611.82,HL,0.0,-1.50,0.75,2.25,-1,2,SATr,1.0,5,noncore
29,2612.97,2612.23,HL,0.0,-0.25,1.25,2.00,-1,5,SATr,1.0,5,noncore
31,2613.21,2612.75,LL,1.0,0.50,1.75,1.00,-1,9,SATr,1.0,5,noncore
33,2612.67,2612.65,LL,1.0,1.75,2.25,0.29,-1,35,SATr,1.0,5,noncore


In [48]:
#
# Plotting diff swing tyoes vs hours - Long swings size (in points)
# 
trace0 = go.Scatter(
    y = df_LWTr['points'],
    x = df_LWTr['hour'],
    name = 'Upswing with Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 255, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 255, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_LATr['points'],
    x = df_LATr['hour'],
    name = 'Upswing Against Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 0, 255, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 255)'
        )
    )
)

layout = go.Layout(autosize=True,
                   title = 'Upswing Size With & Against the Market Trend by Hour',legend=dict(x=.7, y=1.0),
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False),title='Trading Hour'),
                yaxis=go.layout.YAxis(title='Swing Size in Points',automargin=True,autorange=True)) 
data = [trace0, trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Up Swings With & Against Trend by Hour.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Up Swings With & Against Trend by Hour.html'

In [49]:
#
# Plotting diff swing tyoes vs hours - Long  swing size in points - core hours only
# 
trace0 = go.Scatter(
    y = df_LWTr_core['points'],
    x = df_LWTr_core['hour'],
    name = 'Upswing with Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 255, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 255, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_LATr_core['points'],
    x = df_LATr_core['hour'],
    name = 'Upswing Against Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 0, 255, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 255)'
        )
    )
)

layout = go.Layout(autosize=True,
                   title = "Upswing Size With & Against the Market Trend by Hour",legend=dict(x=.2, y=1.0),
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False),title='Core Trading Hours (9am - 3pm EST)'),
                yaxis=go.layout.YAxis(title='Swing Size in Points',automargin=True,autorange=True)) #,title='Swing Length in Points')
data = [trace0, trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Up Swings With & Against Trend by Hour.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Up Swings With & Against Trend by Hour.html'

In [50]:
#
# Plotting diff swing tyoes vs hours - Long swing length vs hour
# 
trace0 = go.Scatter(
    y = df_LWTr['swing_len'],
    x = df_LWTr['hour'],
    name = 'Upswing with Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 255, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 255, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_LATr['swing_len'],
    x = df_LATr['hour'],
    name = 'Upswing Against Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 0, 255, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 255)'
        )
    )
)

layout = go.Layout(autosize=True,
                   title = 'Upswing Length With & Against the Market Trend by Hour',legend=dict(x=.2, y=1.0),
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False), title='Trading Hour'),
                yaxis=go.layout.YAxis(title='Swing Length in Bars',automargin=True,autorange=True))
data = [trace0, trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Long With & Against Trend - Swing Length by Hour.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Long With & Against Trend - Swing Length by Hour.html'

In [51]:
#
# Plotting diff swing tyoes vs hours - Swing Length vs hour - core hours only
# 
trace0 = go.Scatter(
    y = df_LWTr_core['swing_len'],
    x = df_LWTr_core['hour'],
    name = 'Upswing with Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 255, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 255, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_LATr_core['swing_len'],
    x = df_LATr_core['hour'],
    name = 'Upswing Against Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 0, 255, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 255)'
        )
    )
)

layout = go.Layout(autosize=True,
                   title = 'Upswing Length With & Against the Market by Hour',legend=dict(x=.2, y=1.0),
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False), title='Core Trading Hours (9am - 3pm EST)'),
                yaxis=go.layout.YAxis(title='Swing Length in Bars',automargin=True,autorange=True))
data = [trace0, trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Long With & Against Trend - Swing Length by Hour.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Long With & Against Trend - Swing Length by Hour.html'

In [52]:
#
# Plotting diff swing tyoes vs hours - Short swing size in points
# 
trace0 = go.Scatter(
    y = df_SWTr['points'],
    x = df_SWTr['hour'],
    name = 'Downswing with Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(255, 0, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_SATr['points'],
    x = df_SATr['hour'],
    name = 'Downswing Against Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 0, 255, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 255)'
        )
    )
)

layout = go.Layout(autosize=True,
                   title = 'Downswing Size With & Against the Market Trend by Hour',legend=dict(x=.2, y=1.0),
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False),title='Trading Hour'),
                yaxis=go.layout.YAxis(title='Swing Size in Points',automargin=True,autorange=True)) #,title='Swing Length in Points')
data = [trace0, trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Down Swings With & Against Trend by Hour.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Down Swings With & Against Trend by Hour.html'

In [53]:
#
# Plotting diff swing tyoes vs hours - Short swing size in points - Core Hours only
# 
trace0 = go.Scatter(
    y = df_SWTr_core['points'],
    x = df_SWTr_core['hour'],
    name = 'Downswing with Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(255, 0, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_SATr_core['points'],
    x = df_SATr_core['hour'],
    name = 'Downswing Against Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 0, 255, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 255)'
        )
    )
)

layout = go.Layout(autosize=True,
                   title = 'Downswing Size With & Against the Market Trend by Hour',legend=dict(x=.2, y=1.0),
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False),title='Core Trading Hours (9am - 3pm EST)'),
                yaxis=go.layout.YAxis(title='Swing Size in Points',automargin=True,autorange=True)) #,title='Swing Length in Points')
data = [trace0, trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Down Swings With & Against Trend by Hour - Core Hours.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Down Swings With & Against Trend by Hour - Core Hours.html'

In [54]:
#
# Plotting diff swing types vs hours - Short Swing Lengths by hour
# 
trace0 = go.Scatter(
    y = df_SWTr['swing_len'],
    x = df_SWTr['hour'],
    name = 'Downswing with Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(255, 0, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_SATr['swing_len'],
    x = df_SATr['hour'],
    name = 'Downswing Against Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 0, 255, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 255)'
        )
    )
)

layout = go.Layout(autosize=True,
                   title = 'Downswing Length With & Against the Market Trend by Hour',legend=dict(x=.2, y=1.0),
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False),title='Trading Hour'),
                yaxis=go.layout.YAxis(title='Swing Length in Bars',automargin=True,autorange=True)) #,title='Swing Length in Points')
data = [trace0, trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Down Swings With & Against Trend by Hour.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Down Swings With & Against Trend by Hour.html'

In [55]:
#
# Plotting diff swing types vs hours - Short Swing Lengths - Core Hours Only
# 
trace0 = go.Scatter(
    y = df_SWTr_core['swing_len'],
    x = df_SWTr_core['hour'],
    name = 'Downswing with Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(255, 0, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_SATr_core['swing_len'],
    x = df_SATr_core['hour'],
    name = 'Downswing Against Trend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(0, 0, 255, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 255)'
        )
    )
)

layout = go.Layout(autosize=True,
                   title = 'Downswing Length With & Against the Market Trend by Hour',legend=dict(x=.3, y=1.0),
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False),title='Core Trading Hours (9am - 3 pm)'),
                yaxis=go.layout.YAxis(title='Swing Length in Bars',automargin=True,autorange=True)) 
data = [trace0, trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Down Swings With & Against Trend by Hour - Core Hours.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Down Swings With & Against Trend by Hour - Core Hours.html'

In [56]:
#
# Plotting diff swing types - Short Trades
# 
trace0 = go.Scatter(
    y = df_SWTr['points'],
    x = df_SWTr['swing_len'],
    name = 'SWTr',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(152, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_SATr['points'],
    x = df_SATr['swing_len'],
    name = 'SATr',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True))
data = [trace0,trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Short Trades - With and Against Trend.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Short Trades - With and Against Trend.html'

In [57]:
# Distrubution plot - all up swings 
# 
import plotly.figure_factory as ff

import numpy as np

# Add histogram data
x1 = df_LWTr['points'] 
x2 = df_LATr['points']

# Group data together
hist_data = [x1, x2]

group_labels = ['Upswing With Trend', 'Upswing Against Trend']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.5, 1, 1.5, 2.0, 2.5, 3.0, 4.0])
fig['layout'].update(title='Upswing Point Size Distplot')
fig['layout'].update(legend=dict(x=.3, y=1.0))


# Plot
pyo.plot(fig, filename='Long Trades - Distplot with Multiple Bin Sizes.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Long Trades - Distplot with Multiple Bin Sizes.html'

In [58]:
# Distrubution plot - up swings - core hours only
# 

import plotly.figure_factory as ff

import numpy as np

# Add histogram data
x1 = df_LWTr_core['points'] 
x2 = df_LATr_core['points']

# Group data together
hist_data = [x1, x2]

group_labels = ['Upswing With Trend', 'Upswing Against Trend']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.5, 1, 1.5, 2.0, 2.5, 3.0, 4.0])
fig['layout'].update(title='Upswing Point Size Distplot - Core Hours (9am - 3 pm)')
fig['layout'].update(legend=dict(x=.3, y=1.0))

# Plot
pyo.plot(fig, filename='Long Trades - Distplot with Multiple Bin Sizes - Core Hours.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Long Trades - Distplot with Multiple Bin Sizes - Core Hours.html'

In [59]:
# Distrubution plot - up swings - noncore
# 
import plotly.figure_factory as ff

import numpy as np

# Add histogram data
x1 = df_LWTr_noncore['points'] 
x2 = df_LATr_noncore['points']

# Group data together
hist_data = [x1, x2]

group_labels = ['Upswing With Trend', 'Upswing Against Trend']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.5, 1, 1.5, 2.0, 2.5, 3.0, 4.0])
fig['layout'].update(title='Upswing Point Size Distplot - Noncore hours (4pm - 8am)')
fig['layout'].update(legend=dict(x=.3, y=1.0))

# Plot
pyo.plot(fig, filename='Long Trades - Distplot with Multiple Bin Sizes - Noncore Hours.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Long Trades - Distplot with Multiple Bin Sizes - Noncore Hours.html'

In [60]:
# Distrubution plot - all up swings - broken out between core and non-core
# 
import plotly.figure_factory as ff

import numpy as np

# Add histogram data
x1 = df_LWTr_core['points'] 
x2 = df_LATr_core['points']
x3 = df_LWTr_noncore['points'] 
x4 = df_LATr_noncore['points']



# Group data together
hist_data = [x1, x2, x3, x4]

group_labels = ['Upswing With Trend - Core Hours', 'Upswing Against Trend - Core Hours', 'Upswing With Trend - Noncore Hours', 'Upswing Against Trend - Noncore Hours']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.5, 1, 1.5, 2.0, 2.5, 3.0, 4.0])
fig['layout'].update(title='Upswing Point Size Distplot')
fig['layout'].update(legend=dict(x=.3, y=1.0))

# Plot
pyo.plot(fig, filename='Long Trades - Distplot with Multiple Bin Sizes.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Long Trades - Distplot with Multiple Bin Sizes.html'

In [61]:
import plotly.figure_factory as ff

import numpy as np

# Add histogram data
x1 = df_SWTr['points'] 
x2 = df_SATr['points']

# Group data together
hist_data = [x1, x2]

group_labels = ['Downswing With Trend', 'Downswing Against Trend']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.5, 1, 1.5, 2.0, 2.5, 3.0, 4.0])
fig['layout'].update(title='Downswing Point Size Distplot')
fig['layout'].update(legend=dict(x=.3, y=1.0))

# Plot!
pyo.plot(fig, filename='Downswing Trades - Distplot with Multiple Bin Sizes.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Downswing Trades - Distplot with Multiple Bin Sizes.html'

In [62]:
# Distrubution plot - all down swings - broken out between core and non-core
# 
import plotly.figure_factory as ff

import numpy as np

# Add histogram data
x1 = df_SWTr_core['points'] 
x2 = df_SATr_core['points']
x3 = df_SWTr_noncore['points'] 
x4 = df_SATr_noncore['points']



# Group data together
hist_data = [x1, x2, x3, x4]

group_labels = ['Downswing With Trend - Core Hours', 'Downswing Against Trend - Core Hours', 'Downswing With Trend - Noncore Hours', 'Downswing Against Trend - Noncore Hours']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.5, 1, 1.5, 2.0, 2.5, 3.0, 4.0])
fig['layout'].update(title='Downswing Point Size Distplot')
fig['layout'].update(legend=dict(x=.3, y=1.0))
#layout = go.Layout(
#    legend=dict(x=-.1, y=1.2)

# Plot
pyo.plot(fig, filename='Downswings - Distplot with Multiple Bin Sizes.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Downswings - Distplot with Multiple Bin Sizes.html'

# Trend Moves (Long to Short) - MFE and MAE Calcs
### MFE - Most Favorable Excursion
### MAE - Most Adverse Excursion

In [44]:
df_mini.head()

,open,high,low,close,21ema,55ema,trend,trail_hi,trail_hix,trail_lo,trail_lox,hour
Date_Time,,,,,,,,,,,,
2019-01-16 03:53:40,2614.50,2614.50,2614.25,2614.25,2614.344078,2614.128876,1,2614.50,NaN,2614.25,NaN,3
2019-01-16 03:54:00,2614.25,2614.50,2614.25,2614.25,2614.335525,2614.133202,1,2614.50,0.00,2614.25,0.00,3
2019-01-16 03:54:20,2614.25,2614.25,2614.00,2614.25,2614.327750,2614.137373,1,2614.50,-0.25,2614.00,0.25,3
2019-01-16 03:54:40,2614.25,2614.50,2614.25,2614.50,2614.343409,2614.150324,1,2614.50,0.00,2614.00,-0.25,3
2019-01-16 03:55:00,2614.50,2614.75,2614.25,2614.25,2614.334917,2614.153884,1,2614.75,0.25,2614.00,-0.25,3


In [45]:
# 
# Meaasuring Trend Moves (long and short) and calculating MFE (Most Favorable Excursion) and MAE (Most Adverse Excursion)
# 
# Input Data: Date/Time  [0]open, [1]high, [2]low, [3]close, [4]21ema, [5]55ema, [6]trend, [7]trail_hi, [8]trail_hix, [9]trail_lo, [10]trail_lox [11] hour
#
# initial var setup

trend_cnt = 1
trend_up_down = 0
trend = 0
prev_trend = 0
diff = 2.0
trend_mfe = 0
trend_mae = 0
trend_c = 0
trend_prev_mae = 0
uptrend_list = []
downtrend_list = []
all_trend_list = []

debug_print = 0


def trend_up_mae(curr_trend,curr_high,curr_low):         # this is getting called 2x - on new swing begin and end - need to only call it 1x
    global trend_mae, trend_mae_bar, trend_ll, trend_cnt, trend_c_loc, trend_c_prev_loc
    if curr_trend == 1 and trend_ll != 0:
        if trend_c - curr_low > trend_mae:
            trend_mae = (trend_c - curr_low)
            trend_mae_bar = df_mini.index.get_loc(i)
            trend_ll = curr_low
        hold_mfe_len = max(trend_mfe_bar,trend_c_prev_loc) - trend_c_prev_loc
        if hold_mfe_len == 0:
            hold_mae = trend_mae * -1
            hold_mae_len = trend_c_loc - trend_c_prev_loc
        else: 
            hold_mae = max(trend_mae*-1,trend_mae_prev*-1)
            hold_mae_len = min(trend_mae_bar - trend_c_prev_loc, trend_mae_prev_bar - trend_c_prev_loc)
        all_trend_list.append({'TrndNum':trend_cnt, 'Date_Time':str(trend_c_prev_date), 'Trnd':prev_trend, 'HHi': trend_hh, 'Trend_C': trend_c, 'LLow': trend_ll, 'MFE': trend_mfe, \
                            'MAE': hold_mae, 'Prev Trend C Loc': trend_c_prev_loc, 'Trend C Loc': trend_c_loc, 'Length':(trend_c_loc-trend_c_prev_loc), \
                            'MFE Len:': hold_mfe_len, 'MAE Len:': hold_mae_len})                            
        trend_cnt += 1



def trend_dn_mae(curr_trend,curr_high,curr_low):         # this is getting called 2x - on new swing begin and end - need to only call it 1x
    global trend_mae, trend_mae_bar, trend_hh, trend_cnt
    if curr_trend == -1 and trend_hh != 0:
        if curr_high - trend_c > trend_mae:
            trend_mae = (curr_high - trend_c)
            trend_mae_bar = df_mini.index.get_loc(i)
            trend_hh = curr_high
        hold_mfe_len = max(trend_mfe_bar,trend_c_prev_loc) - trend_c_prev_loc
        if hold_mfe_len == 0:
            hold_mae = trend_mae * -1
            hold_mae_len = trend_c_loc - trend_c_prev_loc
        else: 
            hold_mae = max(trend_mae*-1,trend_mae_prev*-1)
            hold_mae_len = min(trend_mae_bar - trend_c_prev_loc, trend_mae_prev_bar - trend_c_prev_loc)
        all_trend_list.append({'TrndNum':trend_cnt, 'Date_Time':str(trend_c_prev_date), 'Trnd':prev_trend, 'HHi': trend_hh, 'Trend_C': trend_c, 'LLow': trend_ll, 'MFE': trend_mfe, \
                            'MAE': hold_mae, 'Prev Trend C Loc': trend_c_prev_loc, 'Trend C Loc': trend_c_loc, 'Length':(trend_c_loc-trend_c_prev_loc), \
                            'MFE Len:': hold_mfe_len, 'MAE Len:': hold_mae_len})
        trend_cnt += 1
        
print(datetime.now())
last_row = len(df_mini)
print(f'Lastrow: {last_row}')

for i, row in df_mini.iterrows():
    if df_mini.index.get_loc(i) == 0:          # initial run through - set entries for first bar
        trend = row[6]
        trend_c = row[3]
        trend_h = row[1]
        trend_c_loc = df_mini.index.get_loc(i)
        swing_h_trend = row[6]
        trend_hh = 0
        trend_ll = 0
        trend_c_date = i
        

    ##--------- Trend Change --------------------------------------------------------------------------------#  
    if prev_trend != row[6] and df_mini.index.get_loc(i) != 0:   # close out MAE - last bar can often be big
        trend_c_prev_loc = trend_c_loc
        trend_c_loc = df_mini.index.get_loc(i)
        trend_c_prev_date = trend_c_date
        trend_c_date = i
        trend_up_mae(trend, row[1], row[2])   # call func when trend changes - calc MAE on last bar
        trend_dn_mae(trend, row[1], row[2])   # call func when trend changes - calc MAE on last bar
        trend = row[6]
        trend_c = row[3]
        prev_trend = row[6]
        trend_mfe = 0
        trend_mfe_bar = 0
        trend_mfe_prev = 0        
        trend_mfe_prev_bar = 0        
        trend_mae = 0
        trend_mae_bar = 0        
        trend_mae_prev = 0
        trend_mae_prev_bar = 0
        trend_hh = 0
        trend_ll = 0
        
    ##--------- UP SWING --------------------------------------------------------------------------------#  
    if trend == 1 and trend_c_loc != df_mini.index.get_loc(i):    # don't calc MFE, MAE until next bar after trend change
        
        if df_mini.index.get_loc(i) - trend_c_loc == 1:           # set MFE to initial bar for rate "no MAE" situations
            trend_mae =  max(0,(trend_c - row[2]))                # check max due to intraday tick price gaps
            trend_mae_bar = df_mini.index.get_loc(i)
            trend_ll = row[2]
        
        if row[1] - trend_c > trend_mfe:
            trend_mfe = row[1] - trend_c
            trend_mfe_bar = df_mini.index.get_loc(i)
            trend_hh = row[1]
            trend_mae_prev = trend_mae
            trend_mae_prev_bar = trend_mae_bar
        
        if (trend_c - row[2]) > trend_mae:
            if df_mini.index.get_loc(i) >= trend_mfe_bar:
                trend_mae = (trend_c - row[2])
                trend_mae_bar = df_mini.index.get_loc(i)
                trend_ll = row[2]
        
    ##--------- DOWN SWING --------------------------------------------------------------------------------#  
    if trend == -1 and trend_c_loc != df_mini.index.get_loc(i):      # don't calc MFE, MAE until next bar after trend change       
        if df_mini.index.get_loc(i) - trend_c_loc == 1:              # set MFE to initial bar for rate "no MAE" situations
            trend_mae = max(0,(row[1] - trend_c))                    # check max due to intraday tick price gaps
            trend_mae_bar = df_mini.index.get_loc(i)
            trend_hh = row[1]
    
        if trend_c - row[2] > trend_mfe:
            trend_mfe = trend_c - row[2]
            trend_mfe_bar = df_mini.index.get_loc(i)
            trend_ll = row[2]
            trend_mae_prev = trend_mae
            trend_mae_prev_bar = trend_mae_bar
        
        if (row[1] - trend_c) > trend_mae:
            if df_mini.index.get_loc(i) >= trend_mfe_bar:
                trend_mae = (row[1] - trend_c)
                trend_mae_bar = df_mini.index.get_loc(i)
                trend_hh = row[1]
    
    ##---------------------------------------------------------------------------------------------------#  
    if df_mini.index.get_loc(i) == 0:     # set up after skipping over initial swing logic
        swing =1

    ##---------------------------------------------------------------------------------------------------#  
    if df_mini.index.get_loc(i) == last_row-1:     # set up after skipping over initial swing logic
        trend_up_mae(trend, row[1], row[2])   # call func when trend changes - calc MAE on last bar
        trend_dn_mae(trend, row[1], row[2])   # call func when trend changes - calc MAE on last bar

        
print(type(all_trend_list))
print(len(all_trend_list))

with open("uptrend_list.txt", "w") as output:
    output.write(str(datetime.now())+"\n")
    for i in range(0,len(all_trend_list)):
        #print('in print loop')
        output.write(str(all_trend_list[i])+"\n")

2019-07-17 07:30:25.149628
Lastrow: 11000
<class 'list'>
93


In [46]:
#
# Converting all trend list to Dataframe for EDA
# 

all_trend_df = pd.DataFrame(all_trend_list)
all_trend_df.head(10)
#test_df.index = pd.to_datetime(test_df.index)
all_trend_df.set_index('Date_Time',inplace=True)
all_trend_df.index = pd.to_datetime(all_trend_df.index)

#df_mini['hour'] = df_mini.index.hour
all_trend_df['hour'] = all_trend_df.index.hour
all_trend_df.head(40)
all_trend_df.to_excel("all_trend_list.xlsx")

In [47]:
all_trend_df.head()

,HHi,LLow,Length,MAE,MAE Len:,MFE,MFE Len:,Prev Trend C Loc,Trend C Loc,Trend_C,Trnd,TrndNum,hour
Date_Time,,,,,,,,,,,,,
2019-01-16 03:54:00,2615.25,2611.5,86,-1.25,7,1.00,33,1,87,2614.25,1.0,1,3
2019-01-16 04:22:40,2613.00,2609.5,148,-0.25,1,2.00,40,87,235,2611.50,-1.0,2,4
2019-01-16 05:12:00,2614.00,2610.5,85,-0.50,2,1.00,15,235,320,2613.00,1.0,3,5
2019-01-16 05:40:20,2611.50,2607.0,348,-0.75,3,3.75,299,320,668,2610.75,-1.0,4,5
2019-01-16 07:36:20,2616.25,2610.0,343,-1.50,10,4.75,284,668,1011,2611.50,1.0,5,7


In [48]:
all_trend_df = pd.merge(all_trend_df,test_core,on='hour',how='left')
all_trend_df.head()

,HHi,LLow,Length,MAE,MAE Len:,MFE,MFE Len:,Prev Trend C Loc,Trend C Loc,Trend_C,Trnd,TrndNum,hour,hour_type
0,2615.25,2611.5,86,-1.25,7,1.00,33,1,87,2614.25,1.0,1,3,noncore
1,2613.00,2609.5,148,-0.25,1,2.00,40,87,235,2611.50,-1.0,2,4,noncore
2,2614.00,2610.5,85,-0.50,2,1.00,15,235,320,2613.00,1.0,3,5,noncore
3,2611.50,2607.0,348,-0.75,3,3.75,299,320,668,2610.75,-1.0,4,5,noncore
4,2616.25,2610.0,343,-1.50,10,4.75,284,668,1011,2611.50,1.0,5,7,noncore


In [49]:
all_trend_df_core = all_trend_df[all_trend_df['hour_type'] == 'core']
all_trend_df_core.head(10)

,HHi,LLow,Length,MAE,MAE Len:,MFE,MFE Len:,Prev Trend C Loc,Trend C Loc,Trend_C,Trnd,TrndNum,hour,hour_type
5,2616.50,2613.75,6,-1.00,1,0.25,1,1011,1017,2614.00,-1.0,6,9,core
6,2624.00,2615.50,149,-0.25,1,8.25,45,1017,1166,2615.75,1.0,7,9,core
7,2622.00,2612.50,88,-1.25,2,8.25,69,1166,1254,2620.75,-1.0,8,10,core
8,2620.00,2616.25,33,-0.75,18,1.50,26,1254,1287,2618.50,1.0,9,10,core
9,2619.25,2613.25,35,-1.00,3,3.25,14,1287,1322,2616.50,-1.0,10,11,core
10,2623.00,2618.00,100,-0.50,3,4.50,64,1322,1422,2618.50,1.0,11,11,core
11,2619.75,2614.75,76,-0.25,1,4.75,29,1422,1498,2619.50,-1.0,12,11,core
12,2619.75,2616.25,7,0.00,1,0.75,1,1498,1505,2619.00,1.0,13,12,core
13,2619.00,2614.00,10,0.00,1,2.50,4,1505,1515,2616.50,-1.0,14,12,core
14,2620.00,2616.00,58,-2.25,9,1.25,12,1515,1573,2618.75,1.0,15,12,core


In [50]:
all_trend_df_noncore = all_trend_df[all_trend_df['hour_type'] == 'noncore']
all_trend_df_noncore.head(10)

,HHi,LLow,Length,MAE,MAE Len:,MFE,MFE Len:,Prev Trend C Loc,Trend C Loc,Trend_C,Trnd,TrndNum,hour,hour_type
0,2615.25,2611.50,86,-1.25,7,1.00,33,1,87,2614.25,1.0,1,3,noncore
1,2613.00,2609.50,148,-0.25,1,2.00,40,87,235,2611.50,-1.0,2,4,noncore
2,2614.00,2610.50,85,-0.50,2,1.00,15,235,320,2613.00,1.0,3,5,noncore
3,2611.50,2607.00,348,-0.75,3,3.75,299,320,668,2610.75,-1.0,4,5,noncore
4,2616.25,2610.00,343,-1.50,10,4.75,284,668,1011,2611.50,1.0,5,7,noncore
24,2614.25,2610.25,138,-2.00,18,2.00,68,2852,2990,2612.25,1.0,25,19,noncore
25,2610.50,2603.25,163,0.00,1,7.25,126,2990,3153,2610.50,-1.0,26,20,noncore
26,2609.00,2605.25,198,-1.25,57,2.50,118,3153,3351,2606.50,1.0,27,21,noncore
27,2608.25,2603.75,361,-2.25,91,2.25,345,3351,3712,2606.00,-1.0,28,22,noncore
28,2606.50,2601.25,219,-2.00,57,0.75,71,3712,3931,2605.75,1.0,29,0,noncore


In [51]:
# Distribution
all_trend_df['MFE'].describe(percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

count    93.000000
mean      3.193548
std       3.764488
min       0.000000
10%       0.250000
20%       0.600000
30%       1.000000
40%       1.250000
50%       2.000000
75%       4.500000
80%       5.000000
90%       7.850000
max      20.750000
Name: MFE, dtype: float64

In [52]:
# Distribution
all_trend_df['MAE'].describe(percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

count    93.000000
mean     -1.241935
std       0.994859
min      -5.000000
10%      -2.500000
20%      -2.000000
30%      -1.500000
40%      -1.250000
50%      -1.000000
75%      -0.500000
80%      -0.350000
90%      -0.250000
max       0.000000
Name: MAE, dtype: float64

In [53]:
# Distribution
all_trend_df_core['MFE'].describe(percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

count    61.000000
mean      3.213115
std       4.063259
min       0.000000
10%       0.250000
20%       0.500000
30%       0.750000
40%       1.250000
50%       1.750000
75%       4.500000
80%       5.000000
90%       8.000000
max      20.750000
Name: MFE, dtype: float64

In [54]:
# Distribution
all_trend_df_noncore['MFE'].describe(percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

count    32.000000
mean      3.156250
std       3.178018
min       0.000000
10%       0.525000
20%       1.000000
30%       1.250000
40%       1.600000
50%       2.000000
75%       4.500000
80%       4.700000
90%       7.225000
max      14.750000
Name: MFE, dtype: float64

In [55]:
# Distribution
all_trend_df_core['MAE'].describe(percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

count    61.000000
mean     -1.159836
std       1.017588
min      -5.000000
10%      -2.500000
20%      -1.750000
30%      -1.250000
40%      -1.250000
50%      -1.000000
75%      -0.500000
80%      -0.250000
90%      -0.250000
max       0.000000
Name: MAE, dtype: float64

In [78]:
# Distribution
all_trend_df_noncore['MAE'].describe(percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

count    32.000000
mean     -1.398438
std       0.945830
min      -3.750000
10%      -2.700000
20%      -2.000000
30%      -1.675000
40%      -1.500000
50%      -1.250000
75%      -0.687500
80%      -0.500000
90%      -0.275000
max       0.000000
Name: MAE, dtype: float64

In [79]:
#
# Plotting trends MFE vs MAE
# 
trace0 = go.Scatter(
    y = all_trend_df['MFE'],
    x = all_trend_df['MFE Len:'],
    name = 'MFE vs MFE Len',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(152, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)
layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True))
data = [trace0]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='All trades - MFE vs MFE Len.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\All trades - MFE vs MFE Len.html'

In [80]:
#
# Plotting trends MFE vs bars
# 
trace0 = go.Scatter(
    y = all_trend_df['MFE'],
    x = all_trend_df['MAE'],
    name = 'MFE vs MAE',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(152, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)
layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True))
#                shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
data = [trace0]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='All trades - MFE vs MAE.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\All trades - MFE vs MAE.html'

In [81]:
#
# Plotting trends MFE vs bars - Core hours
# 
trace0 = go.Scatter(
    y = all_trend_df_core['MFE'],
    x = all_trend_df_core['MAE'],
    name = 'MFE vs MAE',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(152, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)
layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True))
#                shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
data = [trace0]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='All trades - MFE vs MAE.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\All trades - MFE vs MAE.html'

# Machine Learning for Predicting Price Swings Hitting Previous Swing Tops / Bottoms

In [56]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, roc_curve, auc

print("Libraries imported successfully!")

Libraries imported successfully!


In [66]:
# Make a copy of the dataframe
test_df2 = test_df.copy()

In [67]:
test_df2.head()

,21EMA,55E,BrkType,Brkout,Brkpts,points,retrace,swing,swing_len,swing_type,trend,hour,hour_type
0,0.00,0.00,EH,1.0,0.00,0.25,1.00,1,0,LWTr,1.0,3,noncore
1,2614.33,2614.14,LL,1.0,0.25,0.50,1.00,-1,2,SATr,1.0,3,noncore
2,2614.33,2614.15,HH,1.0,0.25,0.75,2.00,1,2,LWTr,1.0,3,noncore
3,2614.28,2614.14,LL,1.0,1.00,1.75,1.50,-1,4,SATr,1.0,3,noncore
4,2613.97,2614.03,LH,0.0,-0.25,1.50,2.33,1,7,LWTr,1.0,3,noncore


In [68]:
# replace all text values with numeric values
test_df2 = test_df2.replace('noncore',0)
test_df2 = test_df2.replace('core',1)
test_df2 = test_df2.replace('LWTr',1)
test_df2 = test_df2.replace('LATr',2)
test_df2 = test_df2.replace('SWTr',3)
test_df2 = test_df2.replace('SATr',4)

In [69]:
# Delete the Breakout type and Breakout points columns - we are trying to predict if the swing will create a breakout
del test_df2['points']
del test_df2['BrkType']
del test_df2['Brkpts']

In [70]:
test_df2.head()

,21EMA,55E,Brkout,retrace,swing,swing_len,swing_type,trend,hour,hour_type
0,0.00,0.00,1.0,1.00,1,0,1,1.0,3,0
1,2614.33,2614.14,1.0,1.00,-1,2,4,1.0,3,0
2,2614.33,2614.15,1.0,2.00,1,2,1,1.0,3,0
3,2614.28,2614.14,1.0,1.50,-1,4,4,1.0,3,0
4,2613.97,2614.03,0.0,2.33,1,7,1,1.0,3,0


In [71]:
# Prep data
# Define the target and feature variables
target = "Brkout"
feature_cols = test_df2.columns[test_df2.columns != target]

# Create X and Y variables
X = test_df2[feature_cols]
Y = test_df[target]

In [72]:
X.head()

,21EMA,55E,retrace,swing,swing_len,swing_type,trend,hour,hour_type
0,0.00,0.00,1.00,1,0,1,1.0,3,0
1,2614.33,2614.14,1.00,-1,2,4,1.0,3,0
2,2614.33,2614.15,2.00,1,2,1,1.0,3,0
3,2614.28,2614.14,1.50,-1,4,4,1.0,3,0
4,2613.97,2614.03,2.33,1,7,1,1.0,3,0


In [73]:
Y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Brkout, dtype: float64

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(764, 9) (764,)
(192, 9) (192,)


In [75]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(X_train, y_train)

C:\Users\522147\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [76]:
preds= clf.predict(X_test)

In [77]:
pd.crosstab(y_test, preds, rownames=['Actual Result'], colnames=['Predicted Result'])

Predicted Result,0.0,1.0
Actual Result,,
0.0,51,35
1.0,30,76


In [78]:
# print evaluation metrics
print("Accuracy: %.2f" % accuracy_score(y_test, preds))
print("Precision: %.2f" % precision_score(y_test, preds))
print("Recall: %.2f" % recall_score(y_test, preds))

Accuracy: 0.66
Precision: 0.68
Recall: 0.72


In [79]:
list(zip(X_train, clf.feature_importances_))

[('21EMA', 0.15948638541868412),
 ('55E', 0.1577503177822181),
 ('retrace', 0.2884033637699281),
 ('swing', 0.013017958901242294),
 ('swing_len', 0.22493128846799432),
 ('swing_type', 0.03241752608626501),
 ('trend', 0.013652509537218837),
 ('hour', 0.09636810831181195),
 ('hour_type', 0.013972541724637317)]

## End of Code